In [1]:
import gc
import os
import pickle
import random
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score as APS
import polars as pl

import gc
import os
import pickle
import random
import joblib
import pandas as pd
# import polars as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import StratifiedKFold
import numpy as np


In [2]:
class Config:
    PREPROCESS = False
    KAGGLE_NOTEBOOK = False
    DEBUG = True
    
    SEED = 42
    EPOCHS = 5
    BATCH_SIZE = 4096
    LR = 1e-3
    WD = 0.05
    PATIENCE = 5
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    NBR_FOLDS = 15
    SELECTED_FOLDS = [0]
    
    

In [3]:
if Config.KAGGLE_NOTEBOOK:
    RAW_DIR = "/kaggle/input/leash-BELKA/"
    PROCESSED_DIR = "/kaggle/input/belka-enc-dataset"
    OUTPUT_DIR = ""
    MODEL_DIR = ""
else:
    RAW_DIR = "../data/raw/"
    PROCESSED_DIR = "../data/processed/"
    OUTPUT_DIR = "../data/result/"
    MODEL_DIR = "../models/"

In [35]:
# h100 なら全部読み込める 300M行
train = pl.read_parquet(os.path.join(PROCESSED_DIR, "train_enc.parquet"))


In [43]:
# shuffle in polars
train.sample(n=len(train),seed=Config.SEED, shuffle=True)

enc0,enc1,enc2,enc3,enc4,enc5,enc6,enc7,enc8,enc9,enc10,enc11,enc12,enc13,enc14,enc15,enc16,enc17,enc18,enc19,enc20,enc21,enc22,enc23,enc24,enc25,enc26,enc27,enc28,enc29,enc30,enc31,enc32,enc33,enc34,enc35,enc36,…,enc108,enc109,enc110,enc111,enc112,enc113,enc114,enc115,enc116,enc117,enc118,enc119,enc120,enc121,enc122,enc123,enc124,enc125,enc126,enc127,enc128,enc129,enc130,enc131,enc132,enc133,enc134,enc135,enc136,enc137,enc138,enc139,enc140,enc141,bind1,bind2,bind3
u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,…,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i64,i64,i64
28,26,8,17,33,29,30,2,32,19,29,8,3,3,5,32,17,8,12,27,12,12,12,17,7,19,12,17,7,19,12,27,19,33,12,27,35,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,26,8,8,29,8,3,5,32,17,33,12,27,35,12,17,33,8,12,18,12,12,12,35,12,18,36,35,18,12,12,35,12,18,8,19,35,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,6,17,26,28,19,17,26,28,19,33,8,27,8,8,8,8,27,8,33,12,27,35,12,17,33,8,8,18,17,28,19,8,8,6,8,18,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
28,26,8,17,33,29,30,2,32,19,29,8,3,3,5,32,17,8,12,27,12,12,12,17,8,1,19,12,17,8,1,19,12,27,19,33,12,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,12,27,12,12,17,33,12,18,35,12,17,33,8,29,8,3,3,5,32,4,28,8,8,33,17,8,19,29,8,3,5,32,4,12,4,12,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
8,33,27,8,8,33,17,8,12,18,12,12,12,17,33,12,4,35,12,17,33,8,8,25,17,12,16,12,12,12,17,8,1,19,12,12,16,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,12,27,12,12,12,12,18,13,12,17,8,8,33,12,4,35,12,17,33,8,8,8,33,25,8,8,33,8,17,26,28,19,8,25,19,35,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,8,28,8,17,26,28,19,12,27,35,12,12,12,12,27,33,12,27,35,12,17,33,12,18,12,12,12,4,35,12,12,35,12,4,12,18,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [48]:
# get rows of wcich bind1 is 1 or bind2 is 1
train_binder= train.filter((pl.col("bind1") == 1) | (pl.col("bind2") == 1) | (pl.col("bind3") == 1))

In [50]:
# get only rows of which bind1, bind2, bind3 are all 0, n_rows = 1000
train_unbinder= train.filter((pl.col("bind1") == 0) & (pl.col("bind2") == 0) & (pl.col("bind3") == 0)).head(len(train_binder))

In [53]:
train_balanced = train_binder.vstack(train_unbinder)
train_balanced = train_balanced.sample(n=len(train_balanced),seed=Config.SEED, shuffle=True)
# save
train_balanced.write_parquet(os.path.join(PROCESSED_DIR, "train_enc_balanced.parquet"))

In [54]:
train_balanced

enc0,enc1,enc2,enc3,enc4,enc5,enc6,enc7,enc8,enc9,enc10,enc11,enc12,enc13,enc14,enc15,enc16,enc17,enc18,enc19,enc20,enc21,enc22,enc23,enc24,enc25,enc26,enc27,enc28,enc29,enc30,enc31,enc32,enc33,enc34,enc35,enc36,…,enc108,enc109,enc110,enc111,enc112,enc113,enc114,enc115,enc116,enc117,enc118,enc119,enc120,enc121,enc122,enc123,enc124,enc125,enc126,enc127,enc128,enc129,enc130,enc131,enc132,enc133,enc134,enc135,enc136,enc137,enc138,enc139,enc140,enc141,bind1,bind2,bind3
u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,…,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i64,i64,i64
8,28,8,17,26,28,19,12,27,12,12,12,17,33,12,18,35,12,17,33,8,8,8,4,8,6,8,4,19,35,12,17,33,29,8,3,5,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8,22,8,8,29,8,3,3,32,17,8,19,17,33,12,27,35,12,17,33,8,8,8,18,28,8,8,8,18,17,8,19,8,19,35,12,17,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,22,8,8,29,8,3,3,5,32,17,8,8,17,26,28,19,33,29,30,2,32,19,33,12,27,35,12,17,33,8,8,17,8,33,17,8,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,8,28,8,17,26,28,19,12,27,35,12,12,12,12,27,33,12,27,35,12,17,33,12,18,12,12,17,28,8,19,12,17,28,8,19,12,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
33,22,8,12,27,12,12,12,12,12,27,33,12,27,35,12,17,33,8,12,18,12,35,12,4,35,18,8,8,28,8,4,19,35,12,17,33,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
8,12,27,12,10,12,12,27,8,33,12,27,35,12,17,33,12,18,12,12,35,12,17,36,12,4,12,12,12,12,12,4,19,12,18,19,35,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8,12,27,12,17,7,19,12,12,12,12,27,33,12,27,35,12,17,33,8,8,18,8,8,8,17,8,17,26,28,19,33,29,30,2,32,19,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8,22,8,8,29,8,3,3,5,32,17,33,12,27,35,12,17,33,12,18,35,12,35,12,17,8,1,19,12,18,28,8,19,35,12,17,33,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [63]:
test = pl.read_parquet(os.path.join(PROCESSED_DIR, "test_enc.parquet"))
test

enc0,enc1,enc2,enc3,enc4,enc5,enc6,enc7,enc8,enc9,enc10,enc11,enc12,enc13,enc14,enc15,enc16,enc17,enc18,enc19,enc20,enc21,enc22,enc23,enc24,enc25,enc26,enc27,enc28,enc29,enc30,enc31,enc32,enc33,enc34,enc35,enc36,…,enc105,enc106,enc107,enc108,enc109,enc110,enc111,enc112,enc113,enc114,enc115,enc116,enc117,enc118,enc119,enc120,enc121,enc122,enc123,enc124,enc125,enc126,enc127,enc128,enc129,enc130,enc131,enc132,enc133,enc134,enc135,enc136,enc137,enc138,enc139,enc140,enc141
u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,…,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8
8,22,8,8,8,8,29,8,3,5,32,17,33,12,27,35,12,17,33,12,18,12,12,12,17,8,26,8,19,12,12,18,19,35,12,17,33,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,22,8,8,8,8,29,8,3,5,32,17,33,12,27,35,12,17,33,12,18,12,12,12,17,8,26,8,19,12,12,18,19,35,12,17,33,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,22,8,8,8,8,29,8,3,5,32,17,33,12,27,35,12,17,33,12,18,12,12,12,17,8,26,8,19,12,12,18,19,35,12,17,33,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,22,8,8,8,8,29,8,3,5,32,17,33,12,27,35,12,17,33,12,18,12,12,12,17,8,26,8,19,12,12,18,19,35,12,17,33,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,22,8,8,8,8,29,8,3,5,32,17,33,12,27,35,12,17,33,12,18,12,12,12,17,8,26,8,19,12,12,18,19,35,12,17,33,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
8,28,8,27,8,8,8,17,8,8,33,12,18,35,12,17,33,12,4,35,13,12,25,12,12,12,17,7,19,12,12,4,25,19,35,12,17,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,28,8,27,8,8,8,17,8,8,33,12,18,35,12,17,33,12,4,35,13,12,25,12,12,12,17,7,19,12,12,4,25,19,35,12,17,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29,33,36,32,26,29,33,14,32,26,33,8,8,8,29,8,3,5,32,17,33,12,27,35,12,17,33,8,12,18,12,12,12,10,18,19,35,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


エンコード済みローカルのテストデータを取得

割合は確認済み

In [4]:
train = pl.read_parquet(os.path.join(PROCESSED_DIR, "train_enc.parquet"))

# データフレームをシャッフル（sampleを使って全体をランダムに取得）
shuffled_df = train.sample(fraction=1.0, with_replacement=False, seed=Config.SEED, shuffle=True)

# 分割の割合を設定
split_ratio = 0.01

# データフレームを分割
split_index = int(len(shuffled_df) * split_ratio)
local_test = shuffled_df[:split_index]
local_train = shuffled_df[split_index:]

bind1 = local_test.filter(pl.col("bind1") == 1).shape[0] / local_test.shape[0]
bind2 = local_test.filter(pl.col("bind2") == 1).shape[0] / local_test.shape[0]
bind3 = local_test.filter(pl.col("bind3") == 1).shape[0] / local_test.shape[0]
print("local_test: ", bind1, bind2, bind3)

bind1 = local_train.filter(pl.col("bind1") == 1).shape[0] / local_train.shape[0]
bind2 = local_train.filter(pl.col("bind2") == 1).shape[0] / local_train.shape[0]
bind3 = local_train.filter(pl.col("bind3") == 1).shape[0] / local_train.shape[0]
print("local_train: ", bind1, bind2, bind3)

# # save
# local_train.write_parquet(os.path.join(PROCESSED_DIR, "local_train_enc.parquet"))
# local_test.write_parquet(os.path.join(PROCESSED_DIR, "local_test_enc.parquet"))

1/bind1, 1/bind2, 1/bind3

local_test:  0.004758391962249887 0.004222907750397294 0.007442925715028918
local_train:  0.004642043010052996 0.004149111846365343 0.007361144379514238


(215.4223900628149, 241.0154358398433, 135.8484426392938)

In [5]:
raw_train = pl.read_parquet(os.path.join(RAW_DIR, "train.parquet"))

In [12]:
assert len(raw_train) == len(train)*3

In [13]:
local_test

enc0,enc1,enc2,enc3,enc4,enc5,enc6,enc7,enc8,enc9,enc10,enc11,enc12,enc13,enc14,enc15,enc16,enc17,enc18,enc19,enc20,enc21,enc22,enc23,enc24,enc25,enc26,enc27,enc28,enc29,enc30,enc31,enc32,enc33,enc34,enc35,enc36,…,enc108,enc109,enc110,enc111,enc112,enc113,enc114,enc115,enc116,enc117,enc118,enc119,enc120,enc121,enc122,enc123,enc124,enc125,enc126,enc127,enc128,enc129,enc130,enc131,enc132,enc133,enc134,enc135,enc136,enc137,enc138,enc139,enc140,enc141,bind1,bind2,bind3
u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,…,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i64,i64,i64
28,26,8,17,33,29,30,2,32,19,29,8,3,3,5,32,17,8,12,27,12,12,12,17,7,19,12,17,7,19,12,27,19,33,12,27,35,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,26,8,8,29,8,3,5,32,17,33,12,27,35,12,17,33,8,12,18,12,12,12,35,12,18,36,35,18,12,12,35,12,18,8,19,35,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,6,17,26,28,19,17,26,28,19,33,8,27,8,8,8,8,27,8,33,12,27,35,12,17,33,8,8,18,17,28,19,8,8,6,8,18,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
28,26,8,17,33,29,30,2,32,19,29,8,3,3,5,32,17,8,12,27,12,12,12,17,8,1,19,12,17,8,1,19,12,27,19,33,12,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,12,27,12,12,17,33,12,18,35,12,17,33,8,29,8,3,3,5,32,4,28,8,8,33,17,8,19,29,8,3,5,32,4,12,4,12,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
8,28,12,27,12,12,17,33,12,18,35,12,17,33,12,4,12,25,12,17,12,12,16,12,4,8,8,8,16,19,8,8,8,25,19,35,12,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,28,12,27,12,12,12,17,8,1,19,12,17,33,12,18,35,12,17,33,12,4,12,12,12,12,17,28,8,19,35,4,19,35,12,17,33,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
28,26,8,17,8,29,8,3,5,32,17,8,12,27,12,12,12,17,7,19,12,12,27,19,33,12,27,35,12,17,33,8,12,18,12,35,12,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


生データのローカルテストデータ取得

In [5]:
train = pl.read_parquet(os.path.join(RAW_DIR, "train.parquet"))


In [7]:

# データフレームをシャッフル（sampleを使って全体をランダムに取得）
shuffled_df = train.sample(fraction=1.0, with_replacement=False, seed=Config.SEED, shuffle=True)

# 分割の割合を設定
split_ratio = 0.01

# データフレームを分割
split_index = int(len(shuffled_df) * split_ratio)
local_test = shuffled_df[:split_index]
local_train = shuffled_df[split_index:]



In [10]:
bind = local_test.filter(pl.col("binds") == 1).shape[0] / local_test.shape[0]
print("local_test: ", bind)
bind = local_train.filter(pl.col("binds") == 1).shape[0] / local_train.shape[0]
print("local_train: ", bind)

local_test:  0.0054032761743734396
local_train:  0.005384821620336283


In [11]:
# save
local_train.write_parquet(os.path.join(PROCESSED_DIR, "local_train.parquet"))
local_test.write_parquet(os.path.join(PROCESSED_DIR, "local_test.parquet"))